# Introduction to EvoXBench

In this notebook, we will demonstrate 
- how to install EvoXBench
- the basics of EvoXBench

**[EvoXBench](https://arxiv.org/abs/2208.04321)** is an efficient platform 
for facilitating neural architecture search (NAS) 
without the requirement of *GPUs* or 
sophisticated deep learning packages, such as *PyTorch, TensorFlow*, etc.

![](https://raw.githubusercontent.com/EMI-Group/evoxbench/main/assets/evoxbench_overview.png)

## 1. Preparation 
Let's perform the following steps to have EvoXBench properly installed. 

First, download the following two files:
- ``database_xxx.zip`` from [Google Drive](https://drive.google.com/file/d/11bQ1paHEWHDnnTPtxs2OyVY_Re-38DiO/view?usp=sharing) or [Baidu NetDisk](https://pan.baidu.com/s/1PwWloA543-81O-GFkA7GKg)
- ``data_xxx.zip`` from [Google Drive](https://drive.google.com/file/d/1fUZtpTjfEQao2unLKaspL8fOq4xdSXt2/view?usp=sharing) or [Baidu NetDisk](https://pan.baidu.com/s/1yopkISKyjbWIHXFV_Op3pg)

Second, unzip these two files and find their paths
- my ``database`` and ``data`` are unzipped to:
```python
    # /Users/luzhicha/Dropbox/2023/github/evoxbench/
    # └─ database/
    # |  |  __init__.py
    # |  |  db.sqlite3
    # |  |  ...
    # |  
    # └─ data/
    #    └─ darts/
    #    └─ mnv3/
    #    └─ ...
```



In [3]:
print('Installing EvoXBench...')
! pip install evoxbench 1>/dev/null

print('Configurating EvoXBench...')
from evoxbench.database.init import config
# make sure you update these two paths accordingly
config("/Users/luzhicha/Dropbox/2023/github/evoxbench/database", 
       "/Users/luzhicha/Dropbox/2023/github/evoxbench/data")

Installing EvoXBench...
Configurating EvoXBench...
Configuration Succeed!


Good! Now we have successfully installed and configured **EvoXBench**. Let's now get started with some quick examples.

# 2.1 How to create a NAS benchmark (search space)

**EvoXBench** currently supports the following seven search spaces

| $\Omega$ | $D$ | $|\Omega|$ | Objectives | Dataset | Supported metrics |
|:-:|:-:|:-:|:-:|:-:|:-:|
| [NB101](https://github.com/google-research/nasbench) | 26 |423K | $f^{e}$, ${f}^{c}$ | CIFAR-10 | IGD & HV |
| [NB201](https://github.com/D-X-Y/NAS-Bench-201) | 6 | 15.6K | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | CIFAR-10 | IGD & HV |
| [NATS](https://github.com/D-X-Y/NATS-Bench) | 5 | 32.8K | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | CIFAR-10 | IGD & HV |
| [DARTS](https://github.com/automl/nasbench301) | 32 | $\sim10^{21}$ | $f^{e}$, ${f}^{c}$ | CIFAR-10 | HV |
| [ResNet-50](https://github.com/mit-han-lab/once-for-all) | 25 | $\sim10^{14}$ | $f^{e}$, ${f}^{c}$ | ImageNet-1K | HV |
| [Transformer](https://github.com/microsoft/Cream/tree/main/AutoFormer) | 34 | $\sim10^{14}$ | $f^{e}$, ${f}^{c}$ | ImageNet-1K | HV |
| [MNV3](https://github.com/mit-han-lab/once-for-all) | 21 | $\sim10^{20}$ | $f^{e}$, ${f}^{c}$, ${f}^{\mathcal{H}}$ | ImageNet-1K | HV |


In [2]:
# NAS-Bench-101 search space
from evoxbench.benchmarks import NASBench101Benchmark
objs = 'err&params'  # ['err&params', 'err&flops', 'err&params&flops']
benchmark = NASBench101Benchmark(objs=objs, normalized_objectives=False)
print("Benchmaking on NB101 search space with objectives: {}".format(objs))

# # NAS-Bench-201 search space
# from evoxbench.benchmarks import NASBench201Benchmark
# objs = 'err&params&flops&latency&energy'
# # ['err&params', 'err&flops', 'err&latency', 'err&params&flops', 'err&params&latency', ...]
# hardware = 'edgegpu'  # ['edgegpu', 'raspi4', 'edgetpu', 'pixel3', 'eyeriss', 'fpga']
# benchmark = NASBench201Benchmark(objs=objs, hardware=hardware, normalized_objectives=False)
# print("Benchmaking on NB201 search space with objectives: {} on hardware device: {}".format(
#     objs, hardware))

# # NATS size search space 
# from evoxbench.benchmarks import NATSBenchmark
# objs = 'err&params&flops&latency'
# # ['err&params', 'err&flops', 'err&latency', 'err&params&flops', 'err&params&latency', ...]
# benchmark = NATSBenchmark(objs=objs, normalized_objectives=False)
# print("Benchmaking on NATS search space with objectives: {}".format(objs))

Auto Configuration Succeed!, Using database /Users/luzhicha/Dropbox/2023/github/evoxbench/database.


ModuleNotFoundError: No module named 'evoxbench.utils'

# 2.2 How to evaluate an architecture

In [ ]:
# let's randomly create N architectures
N = 5
archs = benchmark.search_space.sample(N)
print('Randomly create {} architectures:'.format(N))
print(archs)

In [ ]:
# encode architecture (phenotype) to decision variables (genotypes)
X = benchmark.search_space.encode(archs)
print('Encode architectures to decision variables X: ')
print(X)

In [ ]:
# Evaluate the objective values
# if true_eval is True, return mean TEST accuracy over multiple runs, 
# should onlybe used for final comparison.
true_eval = True
F = benchmark.evaluate(X, true_eval=true_eval)
print("Evaluating architectures for objectives: {}".format(objs))
print(F)

In [ ]:
# Evaluate the objective values
# if true_eval is False, return VALIDATION accuracy from one (randomly selected) run, 
# should be used during search
true_eval = False
print("Evaluating architectures for objectives: {}".format(objs))
for i in range(5):
    F = benchmark.evaluate(X, true_eval=true_eval)
    print("Trial {}:".format(i+1))
    print(F)

In [ ]:
# you may also compute the IGD and HV values for NB101, NB201, NATS search space, 
# where all searchable architectures are exhaustively evaluated.
igd = benchmark.calc_perf_indicator(X, 'igd')
hv = benchmark.calc_perf_indicator(X, 'hv')
print("IGD metric = {}".format(igd))
print("HV metric = {}".format(hv))